# Data

In [1]:
# colab 경로 설정
from google.colab import drive
drive.mount('/content/dirve')
data_path = '/content/dirve/MyDrive/NAVER boostcamp AI Tech 5기/competitions/RecSys 기초대회 (1)/data/대회 데이터/'
v_data_path = data_path + 'v5/'

Mounted at /content/dirve


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
# data_path = '../data/'
# v_data_path='../data/v3/'

path = v_data_path

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
train_ratings = pd.read_csv(path+'train_ratings.csv')
test_ratings = pd.read_csv(path+'test_ratings.csv')

print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('train_ratings shape: ', train_ratings.shape)
print('test_ratings shape: ', test_ratings.shape)

users shape:  (68092, 6)
books shape:  (149570, 11)
train_ratings shape:  (306795, 3)
test_ratings shape:  (76699, 3)


# v6

## 결측치 개수 column 추가

In [27]:
# users['user_count_na'] = users.isna().sum(axis=1) # users는 결측치가 없음
books['book_count_na'] = books.isna().sum(axis=1)

In [28]:
# users.head()

,user_id,age,location_city,location_state,location_country,rating_count,user_count_na
0,8,28,timmins,ontario,canada,7,0
1,11400,49,ottawa,ontario,canada,12,0
2,11676,36,logansport,britishcolumbia,britishcolumbia,5520,0
3,67544,30,toronto,ontario,canada,7,0
4,85526,36,victoria,britishcolumbia,canada,120,0


In [29]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_high,book_count_na
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,others,0
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,Perennial,http://images.amazon.com/images/P/0060973129.0...,en,1940 1949,"Here, for the first time in paperback, is an o...",NaN,others,1
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,medical,"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg,others,0
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,fiction,A Chinese immigrant who is convinced she is dy...,NaN,fiction,1
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,history,"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg,others,0


## 아이템 개수를 값으로 가지는 count_book column 추가

In [30]:
merge = train_ratings.merge(books, how='left', on='isbn')
data = merge.merge(users, how='inner', on='user_id')
print('merge 결과 shape: ', data.shape)

merge 결과 shape:  (306795, 20)


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 306794
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              306795 non-null  int64  
 1   isbn                 306795 non-null  object 
 2   rating               306795 non-null  int64  
 3   book_title           306795 non-null  object 
 4   book_author          306794 non-null  object 
 5   year_of_publication  306795 non-null  float64
 6   publisher            306795 non-null  object 
 7   img_url              306795 non-null  object 
 8   language             306795 non-null  object 
 9   category             185574 non-null  object 
 10  summary              187711 non-null  object 
 11  img_path             257560 non-null  object 
 12  category_high        306795 non-null  object 
 13  book_count_na        306795 non-null  int64  
 14  age                  306795 non-null  int64  
 15  location_city    

In [32]:
book_rating_count = pd.DataFrame(data['isbn'].value_counts()) # rating 된 책의 개수
book_rating_count.reset_index(inplace=True)
book_rating_count.columns = ['isbn', 'book_rating_count']
book_rating_count

,isbn,book_rating_count
0,0316666343,566
1,0971880107,465
2,0385504209,390
3,0312195516,307
4,0060928336,256
...,...,...
129772,0679312544,1
129773,067903515X,1
129774,0676973787,1
129775,0618031685,1


In [33]:
book_rating_mean = pd.DataFrame(data.groupby('isbn')['rating'].mean()) # 책별 평균 rating
book_rating_mean.reset_index(inplace=True)
book_rating_mean.columns = ['isbn', 'book_rating_mean']
book_rating_mean

,isbn,book_rating_mean
0,0000913154,8.0
1,000104687X,6.0
2,0001047213,8.0
3,0001047973,9.0
4,000104799X,6.0
...,...,...
129772,B0001FZGBC,7.0
129773,B0001FZGPI,6.0
129774,B0001FZGRQ,9.0
129775,B0001GMSV2,8.0


In [34]:
books.shape

(149570, 12)

In [35]:
books2 = books.merge(book_rating_count, on='isbn', how='left').merge(book_rating_mean, on='isbn', how='left')
books2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149570 entries, 0 to 149569
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149569 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             149570 non-null  object 
 7   category             80719 non-null   object 
 8   summary              82343 non-null   object 
 9   img_path             107768 non-null  object 
 10  category_high        149570 non-null  object 
 11  book_count_na        149570 non-null  int64  
 12  book_rating_count    129777 non-null  float64
 13  book_rating_mean     129777 non-null  float64
dtypes: float64(3), int64(1), object(10)
memory usage: 17.1+ MB


In [36]:
books2['book_rating_mean'].describe()

count    129777.000000
mean          6.932618
std           2.286600
min           1.000000
25%           6.000000
50%           7.333333
75%           8.500000
max          10.000000
Name: book_rating_mean, dtype: float64

평균 평점 결측값은, 중앙값과  평균을 반올림한 7로 대체

In [37]:
books2['book_rating_count'].fillna(0, inplace=True) # nan인 경우, rating 된 기록이 없는 것이므로 0으로 대체
books2['book_rating_mean'].fillna(7, inplace=True)

In [38]:
books2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149570 entries, 0 to 149569
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149569 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  object 
 5   img_url              149570 non-null  object 
 6   language             149570 non-null  object 
 7   category             80719 non-null   object 
 8   summary              82343 non-null   object 
 9   img_path             107768 non-null  object 
 10  category_high        149570 non-null  object 
 11  book_count_na        149570 non-null  int64  
 12  book_rating_count    149570 non-null  float64
 13  book_rating_mean     149570 non-null  float64
dtypes: float64(3), int64(1), object(10)
memory usage: 17.1+ MB


In [44]:
books2['book_count_na'].value_counts()

0    64856
2    42190
3    25487
1    17036
4        1
Name: book_count_na, dtype: int64

In [45]:
# users.info()

In [41]:
# users.to_csv(data_path + 'v6/users.csv', index=False)
books2.to_csv(data_path + 'v6/books.csv', index=False)